In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.chdir('/Users/awxlong/Desktop/my-studies/hpc_exps/')

In [ ]:
from torchcam.methods import SmoothGradCAMpp
import h5py
import numpy as np
import pickle

In [ ]:
from pytorch_lightning import Trainer
import torch 
from HistoMIL.MODEL.Image.MIL.TransMIL.pl import pl_TransMIL
from HistoMIL.MODEL.Image.MIL.TransMIL.paras import  TransMILParas
from HistoMIL.MODEL.Image.MIL.CLAM.pl import pl_CLAM
from HistoMIL.MODEL.Image.MIL.CLAM.paras import  CLAMParas
from HistoMIL.DATA.Slide.concepts.WholeSlideImage import WholeSlideImageHeatmap


In [ ]:
wsi_object = WholeSlideImageHeatmap(path='/Users/awxlong/Desktop/my-studies/temp_data/CRC/TCGA-CRC/8472de58-9075-4534-b00b-3a87ba2158da/TCGA-AD-6963-01Z-00-DX1.7df2e133-5f24-4c0a-b7f5-5a65fe3420c9.svs')
# pkl_path = '/Users/awxlong/Desktop/my-studies/temp_data/CRC/Tissue/tcga_folder_1.TCGA-A8-A085-01Z-00-DX1.2B52D1B8-5AD4-4BD6-ADF7-9D65B8EE2622.svs.pkl'
pkl_path = '/Users/awxlong/Desktop/my-studies/temp_data/CRC/Tissue/8472de58-9075-4534-b00b-3a87ba2158da.TCGA-AD-6963-01Z-00-DX1.7df2e133-5f24-4c0a-b7f5-5a65fe3420c9.svs.pkl'
with open(pkl_path, 'rb') as f:
    wsi_tissue = pickle.load(f)
wsi_coords = h5py.File('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Patch/224_224/8472de58-9075-4534-b00b-3a87ba2158da.TCGA-AD-6963-01Z-00-DX1.7df2e133-5f24-4c0a-b7f5-5a65fe3420c9.svs.h5')
wsi_coords = wsi_coords['coords']

In [ ]:
wsi_object.contours_tissue = wsi_tissue['tissue']
wsi_object.holes_tissue = wsi_tissue['holes']

In [ ]:
DEFAULT_TRANSMIL_PARAS = TransMILParas()
pl_model = pl_TransMIL(paras=DEFAULT_TRANSMIL_PARAS)
pl_checkpoint = '/Users/awxlong/Desktop/my-studies/hpc_exps/SavedModelsCV5/mil_transmil_uni_32epochs_reruncv=1_epoch=07-auroc_val=0.81.ckpt'
DEFAULT_CLAM_PARAS = CLAMParas()
clam = pl_CLAM(paras=DEFAULT_CLAM_PARAS)
clam_chekpoint = '/Users/awxlong/Desktop/my-studies/hpc_exps/SavedModelsCV5/mil_transmil_uni_32epochs_reruncv=1_epoch=07-auroc_val=0.81.ckpt'
pl_model = pl_model.load_from_checkpoint(pl_checkpoint)
clam = clam.load_from_checkpoint('/Users/awxlong/Desktop/my-studies/hpc_exps/SavedModelsCV5/mil_clam_uni_42epochs_cv5cv=1_epoch=14-auroc_val=0.83.ckpt')

In [ ]:
# cam_extractor = SmoothGradCAMpp(pl_model.model)

In [ ]:
feat_embedding = torch.load('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/uni/8472de58-9075-4534-b00b-3a87ba2158da.TCGA-AD-6963-01Z-00-DX1.7df2e133-5f24-4c0a-b7f5-5a65fe3420c9.svs.pt')
# feat_embedding = torch.load('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/uni/0bdf189f-bfe3-4587-8303-d1905a3822e4.TCGA-F4-6856-01Z-00-DX1.2872c7b5-b94d-4147-ad90-69f88668135a.svs.pt')
# feat_embedding = torch.load('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/uni/35e53d2b-8e11-4e5c-ac45-87ea9a504c4b.TCGA-F5-6812-01Z-00-DX1.47a702ae-cfc6-48ea-8ac0-f9ec194cfb6e.svs.pt')
# feat_embedding = torch.load('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/resnet50/8472de58-9075-4534-b00b-3a87ba2158da.TCGA-AD-6963-01Z-00-DX1.7df2e133-5f24-4c0a-b7f5-5a65fe3420c9.svs.pt')
# feat_embedding = torch.load('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/74a7137c-ea54-477d-a505-2cf147f3cf24.TCGA-AA-A029-01Z-00-DX1.36BA3129-431D-4AE5-98E6-BA064D0B5062.svs.pt')
# feat_embedding = torch.load('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/prov-gigapath/1ce88052-f889-4630-871d-09de5c5ad369.TCGA-NH-A6GB-01Z-00-DX1.AD90C375-54ED-4EE4-A537-59A2E3FE4BCD.svs.pt')



In [ ]:
print(feat_embedding.shape)
feat_embedding[0:3, :]

In [ ]:
k = 1
with torch.no_grad():
    logits, Y_prob, Y_hat, A, _  = clam.model(feat_embedding[None])
    Y_hat = Y_hat.item()
    A = A.view(-1, 1).cpu().numpy() 
    probs, ids = torch.topk(Y_prob, k)
    probs = probs[-1].cpu().numpy()
    ids = ids[-1].cpu().numpy()
    # return ids, preds_str, probs, A as Y_hats, Y_hats_str, Y_probs, A 
    Y_hats, Y_probs, A = ids, probs, A

# if not os.path.isfile(block_map_save_path): 
            # SAVE ATTENTION SCORES WITH COORDS
    # file = h5py.File(h5_path, "r")
    # coords = file['coords'][:]
    # file.close()
    # asset_dict = {'attention_scores': A, 'coords': coords}
    # block_map_save_path = save_hdf5(block_map_save_path, asset_dict, mode='w')


In [ ]:
patch_size = (224, 224)
wsi_ref_downsample = wsi_object.level_downsamples[0]
patch_custom_downsample = 1
vis_patch_size = tuple((np.array(patch_size) * np.array(wsi_ref_downsample) * patch_custom_downsample ).astype(int))
vis_patch_size
overlap = 0
top_left = None
bot_right = None

In [ ]:
heatmap_vis_args = {'convert_to_percentiles': True, 'blur': True, 'custom_downsample': 1}

In [ ]:
heatmap = wsi_object.visHeatmap(scores=A, coords=wsi_coords, vis_level=1,  
                                cmap='jet', alpha=0.4, **heatmap_vis_args, 
                                binarize=False, blank_canvas=False,
                                thresh=-1,  patch_size = vis_patch_size,
                                overlap=overlap, top_left=top_left, bot_right = bot_right)

In [ ]:
heatmap.save(os.path.join('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Heatmap', 'test_heatmap_uni_orig_res_gauss_blur_contour_tissue.jpg'), quality=100)

### uni

### prov-gigapath

In [ ]:
feat_embedding = torch.load('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/prov-gigapath/0b9fadfc-f3ba-4af2-899c-bf804369fd55.TCGA-A6-A567-01Z-00-DX1.F941874E-9BF7-4E8B-908C-41A638D62275.svs.pt')
feat_embedding

In [ ]:
feat_embedding = torch.load('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/prov-gigapath/1ce88052-f889-4630-871d-09de5c5ad369.TCGA-NH-A6GB-01Z-00-DX1.AD90C375-54ED-4EE4-A537-59A2E3FE4BCD.svs.pt')
feat_embedding

In [ ]:
feat_embedding = torch.load('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/prov-gigapath/fbde56ab-b131-436e-8a6d-24c1e05217ca.TCGA-CM-4743-01Z-00-DX1.f54a6355-5623-498c-96b9-2ff1de6576c6.svs.pt')
feat_embedding

In [ ]:
feat_embedding = torch.load('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/prov-gigapath/0acc3fcf-1209-4824-8d25-e4d0c0f2ac1b.TCGA-A6-2674-01Z-00-DX1.d301f1f5-6f4a-49e6-9c93-f4e8b7f616b8.svs.pt')
feat_embedding

### resnet50

In [ ]:
feat_embedding = torch.load('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/resnet50/0acc3fcf-1209-4824-8d25-e4d0c0f2ac1b.TCGA-A6-2674-01Z-00-DX1.d301f1f5-6f4a-49e6-9c93-f4e8b7f616b8.svs.pt')
feat_embedding[42]

In [ ]:
feat_embedding = torch.load('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/resnet50/0ae1ce57-7580-48de-9f8b-2eff2173b538.TCGA-AH-6903-01Z-00-DX1.29f3feae-e757-4138-9f86-439fac247009.svs.pt')
feat_embedding[42]

In [ ]:
feat_embedding = torch.load('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/resnet50/0b55ba87-0994-4679-9d1c-6cc3ddfd3078.TCGA-A6-5662-01Z-00-DX1.82569684-1c31-4346-af9b-c296a020f624.svs.pt')
feat_embedding[42]